In [1]:
#Import dependencies
import requests
import pandas as pd
import re

# Astronauts Cleanup

In [2]:
#Read csv
file = 'astronauts.csv'
astro_df = pd.read_csv(file)
astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
0,Joseph M. Acaba,2004.0,19.0,Active,5/17/1967,"Inglewood, CA",Male,University of California-Santa Barbara; Univer...,Geology,Geology,NaN,NaN,2,3307,2,13.0,"STS-119 (Discovery), ISS-31/32 (Soyuz)",NaN,NaN
1,Loren W. Acton,NaN,NaN,Retired,3/7/1936,"Lewiston, MT",Male,Montana State University; University of Colorado,Engineering Physics,Solar Physics,NaN,NaN,1,190,0,0.0,STS 51-F (Challenger),NaN,NaN
2,James C. Adamson,1984.0,10.0,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987.0,12.0,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963.0,3.0,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN


In [3]:
#Remove all years greater than 1991
astro_df = astro_df[astro_df['Year'] <= 1991]
astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
2,James C. Adamson,1984.0,10.0,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987.0,12.0,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963.0,3.0,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN
5,Andrew M. Allen,1987.0,12.0,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN
6,Joseph P. Allen,1967.0,6.0,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN


In [4]:
#Change floats to int
astro_df["Year"] = astro_df["Year"].astype(int)
astro_df["Group"] = astro_df["Group"].astype(int)

astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
2,James C. Adamson,1984,10,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987,12,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963,3,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN
5,Andrew M. Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN
6,Joseph P. Allen,1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN


In [5]:
#Strip 'jr' from last name
def strip_jr(name):
    if "Jr." in name:
        result = name[:-4]
    else:
        result = name
    return (result)

astro_df['Name'] = astro_df['Name'].apply(strip_jr)
astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
2,James C. Adamson,1984,10,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987,12,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963,3,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN
5,Andrew M. Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN
6,Joseph P. Allen,1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN


In [6]:
#Split functions
def split_first(astro_string):
    if "." in astro_string:
        if "." in astro_string.split(".")[1]:
            first_name = astro_string.split('.')[0] + "." + astro_string.split('.')[0] + "."
        else:
            first_name = astro_string.split('.')[0] + "."
    else:
        first_name = astro_string.split()[0]
    return(first_name)
        
def split_last(astro_string):
    if "." in astro_string:
        last_name = astro_string.split('.')[1]
        last_name = last_name.strip()
    else:
        last_name = astro_string.split()[1]
    return(last_name)

#Assign to new columns
astro_df['First Name'] = astro_df['Name'].apply(split_first)
astro_df['Last Name'] = astro_df['Name'].apply(split_last)


#Rename Name to Full-Name
astro_df = astro_df.rename(columns={"Name":"Full Name"})
astro_df.head()

,Full Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,...,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission,First Name,Last Name
2,James C. Adamson,1984,10,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,...,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN,James C.,Adamson
3,Thomas D. Akers,1987,12,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,...,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN,Thomas D.,Akers
4,Buzz Aldrin,1963,3,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,...,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN,Buzz,Aldrin
5,Andrew M. Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,...,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN,Andrew M.,Allen
6,Joseph P. Allen,1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,...,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN,Joseph P.,Allen


In [7]:
#Reorder Columns
f_column = astro_df.pop('First Name')
l_column = astro_df.pop('Last Name')
mission_column = astro_df.pop('Missions')
astro_df.insert(1, f_column.name, f_column)
astro_df.insert(2, l_column.name, l_column)
astro_df.insert(3, mission_column.name, mission_column)
astro_df.head()

,Full Name,First Name,Last Name,Missions,Year,Group,Status,Birth Date,Birth Place,Gender,...,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
2,James C. Adamson,James C.,Adamson,"STS-28 (Columbia), STS-43 (Atlantis)",1984,10,Retired,3/3/1946,"Warsaw, NY",Male,...,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,NaN,NaN
3,Thomas D. Akers,Thomas D.,Akers,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",1987,12,Retired,5/20/1951,"St. Louis, MO",Male,...,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,NaN,NaN
4,Buzz Aldrin,Buzz,Aldrin,"Gemini 12, Apollo 11",1963,3,Retired,1/20/1930,"Montclair, NJ",Male,...,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,NaN,NaN
5,Andrew M. Allen,Andrew M.,Allen,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,...,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,NaN,NaN
6,Joseph P. Allen,Joseph P.,Allen,"ST-5 (Columbia), STS 51-A (Discovery)",1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,...,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,NaN,NaN


In [8]:
#Remove all astronauts who didn't pursue missions
astro_df = astro_df[astro_df['Missions'].notnull()]
astro_df.head()

,Full Name,First Name,Last Name,Missions,Year,Group,Status,Birth Date,Birth Place,Gender,...,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
2,James C. Adamson,James C.,Adamson,"STS-28 (Columbia), STS-43 (Atlantis)",1984,10,Retired,3/3/1946,"Warsaw, NY",Male,...,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,NaN,NaN
3,Thomas D. Akers,Thomas D.,Akers,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",1987,12,Retired,5/20/1951,"St. Louis, MO",Male,...,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,NaN,NaN
4,Buzz Aldrin,Buzz,Aldrin,"Gemini 12, Apollo 11",1963,3,Retired,1/20/1930,"Montclair, NJ",Male,...,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,NaN,NaN
5,Andrew M. Allen,Andrew M.,Allen,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,...,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,NaN,NaN
6,Joseph P. Allen,Joseph P.,Allen,"ST-5 (Columbia), STS 51-A (Discovery)",1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,...,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,NaN,NaN


In [11]:
#Setting up name split per mission
df_astro = pd.DataFrame(columns=astro_df.columns)

#Iterate over df in order to create new df, split names appropriately so each mission is seperate
for index, row in astro_df.iterrows():
    check = row["Missions"].split(",")
    
    for mission in check:
        simp_df = pd.DataFrame([[row['Full Name'], row['First Name'], row['Last Name'], mission, row['Year'], row['Group'],
        row['Status'], row['Birth Date'], row['Birth Place'], row['Gender'], row['Alma Mater'],
        row['Undergraduate Major'], row['Graduate Major'], row['Military Rank'],
        row['Military Branch'], row['Space Flights'], row['Space Flight (hr)'], row['Space Walks'],
        row['Space Walks (hr)'], row['Death Date'], row['Death Mission']]], columns=df_astro.columns)
        df_astro = df_astro.append(simp_df)

#Apply sort based on Crew Member's Name
df_astro = df_astro.sort_values(by=["Full Name"])

#Reset index
df_astro = df_astro.reset_index(drop=True)

df_astro.head()

,Full Name,First Name,Last Name,Missions,Year,Group,Status,Birth Date,Birth Place,Gender,...,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
0,Alan B. Shepard,Alan B.,Shepard,Apollo 14,1959,1,Deceased,11/18/1923,"East Derry, NH",Male,...,Naval Sciences,NaN,Rear Admiral,US Navy (Retired),2,216,2,9.0,7/21/1998,NaN
1,Alan B. Shepard,Alan B.,Shepard,Mercury 3,1959,1,Deceased,11/18/1923,"East Derry, NH",Male,...,Naval Sciences,NaN,Rear Admiral,US Navy (Retired),2,216,2,9.0,7/21/1998,NaN
2,Alan L. Bean,Alan L.,Bean,Skylab 3,1963,3,Retired,3/15/1932,"Wheeler, TX",Male,...,Aeronautical Engineering,NaN,Captain,US Navy (Retired),2,1671,3,10.0,NaN,NaN
3,Alan L. Bean,Alan L.,Bean,Apollo 12,1963,3,Retired,3/15/1932,"Wheeler, TX",Male,...,Aeronautical Engineering,NaN,Captain,US Navy (Retired),2,1671,3,10.0,NaN,NaN
4,Alfred M. Worden,Alfred M.,Worden,Apollo 15,1966,5,Retired,2/7/1932,"Jackson, MI",Male,...,Military Science,Aeronautical & Astronautical Engineering,Colonel,US Air Force (Retired),1,295,1,0.5,NaN,NaN


In [12]:
#Clear out tail end of name for Missions
def neaten_mission(mission):
    result = mission.split('(')[0]
    return(result)


df_astro['Missions'] = df_astro['Missions'].apply(neaten_mission)
df_astro.head()

,Full Name,First Name,Last Name,Missions,Year,Group,Status,Birth Date,Birth Place,Gender,...,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
0,Alan B. Shepard,Alan B.,Shepard,Apollo 14,1959,1,Deceased,11/18/1923,"East Derry, NH",Male,...,Naval Sciences,NaN,Rear Admiral,US Navy (Retired),2,216,2,9.0,7/21/1998,NaN
1,Alan B. Shepard,Alan B.,Shepard,Mercury 3,1959,1,Deceased,11/18/1923,"East Derry, NH",Male,...,Naval Sciences,NaN,Rear Admiral,US Navy (Retired),2,216,2,9.0,7/21/1998,NaN
2,Alan L. Bean,Alan L.,Bean,Skylab 3,1963,3,Retired,3/15/1932,"Wheeler, TX",Male,...,Aeronautical Engineering,NaN,Captain,US Navy (Retired),2,1671,3,10.0,NaN,NaN
3,Alan L. Bean,Alan L.,Bean,Apollo 12,1963,3,Retired,3/15/1932,"Wheeler, TX",Male,...,Aeronautical Engineering,NaN,Captain,US Navy (Retired),2,1671,3,10.0,NaN,NaN
4,Alfred M. Worden,Alfred M.,Worden,Apollo 15,1966,5,Retired,2/7/1932,"Jackson, MI",Male,...,Military Science,Aeronautical & Astronautical Engineering,Colonel,US Air Force (Retired),1,295,1,0.5,NaN,NaN


# Mission Cleanup

In [13]:
#Set up url
url = 'https://www.windows2universe.org/space_missions/manned_table.html'

#Retrieve page
tables = pd.read_html(url)

#Create df
df_mission = tables[0]

df_mission.head()

,0,1,2,3,4
0,Mission(Country),Craft,Launch Date,Crew,Mission Highlights
1,Vostok-1(USSR),Kedr (Cedar),"April 12, 1961",Gagarin,Cosmonaut Yuri Garagin became the first human ...
2,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
3,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
4,Vostok-2(USSR),Orel (Eagle),"August 6, 1961",Titov,Titov was the first to spend an entire day in ...


In [14]:
#Set up headers
df_mission.columns = df_mission.iloc[0]
df_mission = df_mission.drop(0)

#Delete "(Country)" from first column header
df_mission = df_mission.rename(columns={"Mission(Country)":"Mission"})

#Delete all rows with "(USSR)" in Mission name
df_mission = df_mission[~df_mission['Mission'].str.contains('(USSR)')]

df_mission.head()

C:\Users\Elizabeth Jaye\acaconda\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


,Mission,Craft,Launch Date,Crew,Mission Highlights
2,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
3,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
5,Mercury-6(USA),Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...
6,Mercury-7(USA),Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...
9,Mercury-8(USA),Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...


In [15]:
#Reset Index
df_mission = df_mission.reset_index(drop=True)
df_mission.head()

,Mission,Craft,Launch Date,Crew,Mission Highlights
0,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
1,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
2,Mercury-6(USA),Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...
3,Mercury-7(USA),Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...
4,Mercury-8(USA),Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...


In [16]:
#Delete country naming
df_mission['Mission'] = df_mission['Mission'].str[:-5]
df_mission.head()

,Mission,Craft,Launch Date,Crew,Mission Highlights
0,Mercury-3,Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
1,Mercury-4,Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
2,Mercury-6,Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...
3,Mercury-7,Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...
4,Mercury-8,Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...


In [17]:
#Remove leading characters and commas from crew column (currently they are only leading, not deliminating)
df_mission['Crew'] = df_mission['Crew'].str.strip(',')
df_mission.head()

,Mission,Craft,Launch Date,Crew,Mission Highlights
0,Mercury-3,Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
1,Mercury-4,Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
2,Mercury-6,Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...
3,Mercury-7,Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...
4,Mercury-8,Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...


In [18]:

#Split names for sorting
def split_it(crew_string):
    result = re.sub(r"(\w)([A-Z])", r"\1 \2", crew_string)

    if 'Mac' in result:
        result = result.replace("Mac ", "Mac")
    
    if "Mc" in result:
        result = result.replace("Mc ", "Mc")
        
    if "van den Berg" in result:
        result = "vandenBerg"
    
    if "van Hoften" in result:
        result = "vanHoften"

    return(result)

df_mission['Crew'] = df_mission['Crew'].apply(split_it)
df_mission.head()

,Mission,Craft,Launch Date,Crew,Mission Highlights
0,Mercury-3,Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
1,Mercury-4,Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
2,Mercury-6,Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...
3,Mercury-7,Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...
4,Mercury-8,Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...


In [19]:
#Move Crew column to second column
my_column = df_mission.pop('Crew')
df_mission.insert(0, my_column.name, my_column)
df_mission.head()

,Crew,Mission,Craft,Launch Date,Mission Highlights
0,Shepard,Mercury-3,Freedom 7,"May 5, 1961",Alan Shepard became the first American in spac...
1,Grissom,Mercury-4,Liberty Bell-7,"July 21, 1961","The second US sub-orbital flight, reaching an ..."
2,Glenn,Mercury-6,Friendship 7,"February 20,1962",The first US manned orbital flight at an orbit...
3,Carpenter,Mercury-7,Aurora 7,"May 24, 1962",The second US manned orbital flight which orbi...
4,Schirra,Mercury-8,Sigma 7,"October 3, 1962",Walter Schirra orbited the Earth six times dur...


In [20]:
#Setting up name split per mission
mission_df = pd.DataFrame(columns=df_mission.columns)
mission_df

#Iterate over df in order to create new df, split names appropriately so each crew member is seperate
for index, row in df_mission.iterrows():
    check = row["Crew"].split(" ")
    
    for name in check:
        mission = row["Mission"]
        craft = row["Craft"]
        launchdate= row["Launch Date"]
        missionhighlights = row["Mission Highlights"]
        simp_df = pd.DataFrame([[name, mission, craft, launchdate, missionhighlights]], columns=mission_df.columns)
        mission_df = mission_df.append(simp_df)

#Apply sort based on Crew Member's Last Name
mission_df = mission_df.sort_values(by=["Crew"])

#Reset index
mission_df = mission_df.reset_index(drop=True)

mission_df.head()

,Crew,Mission,Craft,Launch Date,Mission Highlights
0,Acton,51-F,"Challenger, Spacelab","July 29, 1985",Spacelab 2 carried experiments in life science...
1,Adamson,STS-28,Columbia,"August 13, 1989",The fourth classified Department of Defense Sh...
2,Akers,STS-41,Discovery,"October 6, 1990","Deployed the Ulysses space probe, sending it o..."
3,Al-Saud,51-G,Discovery,"June 17, 1985",The first tri-national space crew deployed thr...
4,Aldrin,Gemini-Titan 12,Gemini 12,"November 11, 1966","In this last Gemini mission, Aldrin made three..."


In [21]:
#Final Name fix for the "van" leaders
def fix_name(crew_name):
    if "vandenBerg" in crew_name:
        name = "van den Berg"  
    elif "vanHoften" in crew_name:
        name = "van Hoften"
    else:
        name = crew_name
        
    return(name)

mission_df['Crew'] = mission_df['Crew'].apply(fix_name)
mission_df.head()

,Crew,Mission,Craft,Launch Date,Mission Highlights
0,Acton,51-F,"Challenger, Spacelab","July 29, 1985",Spacelab 2 carried experiments in life science...
1,Adamson,STS-28,Columbia,"August 13, 1989",The fourth classified Department of Defense Sh...
2,Akers,STS-41,Discovery,"October 6, 1990","Deployed the Ulysses space probe, sending it o..."
3,Al-Saud,51-G,Discovery,"June 17, 1985",The first tri-national space crew deployed thr...
4,Aldrin,Gemini-Titan 12,Gemini 12,"November 11, 1966","In this last Gemini mission, Aldrin made three..."


# Spaceship Cleanup

In [22]:
# Used Pandas to retrieve data from table in html rather than BeautifulSoup
# First table of Orbital crewed spaceship
df1 = (pd.read_html('https://en.wikipedia.org/wiki/Comparison_of_crewed_space_vehicles'))[0]
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*
1,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1]
2,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3]
3,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3]
4,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6]


In [23]:
# Gets first row as a list to use as new header
header_1 = df1.iloc[0]
header_1

0                  Spacecraft
1                      Origin
2                Manufacturer
3                       Range
4                Launchsystem
5                    Crewsize
6                  Length (m)
7                Diameter (m)
8            Launch mass (kg)
9                 Powersystem
10    Payloadto/from ISS (kg)
11          Firstspaceflight*
12            Lastspaceflight
13                   Flights*
Name: 0, dtype: object

In [24]:
# Reset df to drop first row with header names
df1 = df1[1:]
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1]
2,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3]
3,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3]
4,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6]
5,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7]


In [26]:
#Rename columns for proper labels
df1.rename(columns=header_1, inplace=True)
df1.head()

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*
1,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1]
2,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3]
3,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3]
4,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6]
5,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7]


In [27]:
# Add orbital type that separates table subjects
df1['Orbit'] = "Orbital"

In [28]:
# Second table of Sub-orbital crewed spaceship
df2 = (pd.read_html('https://en.wikipedia.org/wiki/Comparison_of_crewed_space_vehicles'))[0]
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*
1,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1]
2,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3]
3,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3]
4,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6]


In [29]:
#Set Column names
df2 = df2.rename(columns=df2.iloc[0]).drop(df2.index[0])
df2.head()

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*
1,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1]
2,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3]
3,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3]
4,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6]
5,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7]


In [30]:
#Rename Orbit type
df2['Orbit'] = "Sub-Orbital"

In [31]:
# See which columns can be joined on
list(df1.columns)

['Spacecraft',
 'Origin',
 'Manufacturer',
 'Range',
 'Launchsystem',
 'Crewsize',
 'Length (m)',
 'Diameter (m)',
 'Launch mass (kg)',
 'Powersystem',
 'Payloadto/from ISS (kg)',
 'Firstspaceflight*',
 'Lastspaceflight',
 'Flights*',
 'Orbit']

In [32]:
# See which columns can be joined on
list(df2.columns)

['Spacecraft',
 'Origin',
 'Manufacturer',
 'Range',
 'Launchsystem',
 'Crewsize',
 'Length (m)',
 'Diameter (m)',
 'Launch mass (kg)',
 'Powersystem',
 'Payloadto/from ISS (kg)',
 'Firstspaceflight*',
 'Lastspaceflight',
 'Flights*',
 'Orbit']

In [34]:
#DF Merge
df3 = df1.merge(df2,how='outer')
df3

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*,Orbit
0,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Orbital
1,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3],Orbital
2,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3],Orbital
3,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6],Orbital
4,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7],Orbital
5,Shenzhou,China,China Academy of Space Technology,LEO,Chang Zheng 2F,3,9.25,2.80,7840,Solar panels,NaN,2003 (1999),Active,7(4),Orbital
6,Gaganyaan,India,Hindustan Aeronautics Limited,LEO,GSLV Mk III,3,7,3,7800,Solar Panels,NaN,2021,NaN,0,Orbital
7,Soyuz 7K-T,Soviet Union,OKB-1,LEO,SoyuzSoyuz-U,2,7.48,2.72,NaN,Batteries,NaN,1973,1981,26 (4)[note 8],Orbital
8,Voskhod,Soviet Union,OKB-1,LEO,Voskhod,3[note 9],5,2.4,NaN,Batteries,NaN,1964 (1964),1965,2 (3),Orbital
9,Vostok,Soviet Union,OKB-1,LEOfirst,Vostok-K,1,4.4,2.43,4725,Batteries,NaN,1961 (1960),1963,6 (7)[note 10],Orbital


In [35]:
# Extract only US ships
ship_df = df3[df3.Origin == 'USA']
ship_df.head()

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*,Orbit
0,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Orbital
1,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3],Orbital
2,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3],Orbital
3,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6],Orbital
4,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7],Orbital


In [37]:
# Rename Launchsytem, Crewsize, Powersystem, Payloadto/from ISS (kg), Firstspaceflight*, Lastspaceflight, Flights* fields
ship_df = ship_df.rename(columns={"Launchsystem":"Launch System", "Crewsize":"Crew Size", "Powersystem":"Power System", "Payloadto/from ISS (kg)":"Payload to/from ISS (kg)", "Firstspaceflight*":"First Space Flight","Lastspaceflight":"Last Space Flight", "Flights*":"Flights"})
ship_df.head()

,Spacecraft,Origin,Manufacturer,Range,Launch System,Crew Size,Length (m),Diameter (m),Launch mass (kg),Power System,Payload to/from ISS (kg),First Space Flight,Last Space Flight,Flights,Orbit
0,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Orbital
1,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3],Orbital
2,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3],Orbital
3,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6],Orbital
4,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7],Orbital


In [38]:
# Reset index
ship_df.reset_index(drop=True,inplace=True)
ship_df

,Spacecraft,Origin,Manufacturer,Range,Launch System,Crew Size,Length (m),Diameter (m),Launch mass (kg),Power System,Payload to/from ISS (kg),First Space Flight,Last Space Flight,Flights,Orbit
0,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Orbital
1,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3],Orbital
2,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3],Orbital
3,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6],Orbital
4,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7],Orbital
5,Biconic Space Vehicle,USA,Blue Origin,LEO,New Glenn,NaN,NaN,NaN,NaN,NaN,NaN,Planned date not known,NaN,0,Orbital
6,CST-100 Starliner,USA,Boeing,LEO,"multiple, initially Atlas V",7 [1],5.03 [2],4.56 [2],NaN,Solar panels,NaN,Planned: 2019,NaN,0,Orbital
7,Dream Chaser,USA,Sierra Nevada Corporation,LEO,"multiple, initially Atlas V",7 [3][4],9 [5],NaN,"11,300 [6]",NaN,NaN,Planned date not known,NaN,0,Orbital
8,Dragon 2,USA,SpaceX,LEO,Falcon 9,7[note 14],8.1[7],3.7[8],"12,055 kg(26,577 lb)[9]",Solar Panels,"3,307/2,507",Planned: 2019 (2019),Active,0(1),Orbital
9,Starship,USA,SpaceX,Solar System[10][note 15],BFR[10],100[note 16],55[10],9[10][note 17],"1,335,000[10]",Solar Panels,NaN,planned (2020) [10]planned 2023,NaN,0,Orbital


In [39]:
# Remove square andd round bracketed notes from dataset
ship_df.replace("\[.*\]", "",regex=True, inplace=True)
ship_df.replace("\(.*\)", "",regex=True, inplace=True)
ship_df.head()

,Spacecraft,Origin,Manufacturer,Range,Launch System,Crew Size,Length (m),Diameter (m),Launch mass (kg),Power System,Payload to/from ISS (kg),First Space Flight,Last Space Flight,Flights,Orbit
0,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961,1963,6,Orbital
1,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC,2,5.56,3.05,NaN,Fuel cells,NaN,1965,1966,10,Orbital
2,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967,1975,15,Orbital
3,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8,37.24,4.8,109000,Fuel cells,"12,500/16,000",1981,2011,135,Orbital
4,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021,NaN,0,Orbital
